In [31]:
from numba import jit, autojit, double
import time as ttime
import numpy as np

In [18]:
@jit(nopython=True)
def f(x, y):
    # A somewhat trivial example
    return x * y

In [19]:
%%timeit -n5
pass
f(1134567, 456890)

The slowest run took 16723.51 times longer than the fastest. This could mean that an intermediate result is being cached 
5 loops, best of 3: 285 ns per loop


In [20]:
f(1j, 2)

2j

In [21]:
def ff(x, y):
    # A somewhat trivial example
    return x * y

In [22]:
%%timeit -n5
pass
ff(1134567, 456890)

5 loops, best of 3: 198 ns per loop


In [23]:
def filter2d(image, filt):
    M, N = image.shape
    Mf, Nf = filt.shape
    Mf2 = Mf // 2
    Nf2 = Nf // 2
    result = np.zeros_like(image)
    for i in range(Mf2, M - Mf2):
        for j in range(Nf2, N - Nf2):
            num = 0.0
            for ii in range(Mf):
                for jj in range(Nf):
                    num += (filt[Mf-1-ii, Nf-1-jj] * image[i-Mf2+ii, j-Nf2+jj])
            result[i, j] = num
    return result

In [24]:
fastfilter_2d = jit(double[:,:](double[:,:], double[:,:]))(filter2d)

In [26]:

# Now fastfilter_2d runs at speeds as if you had first translated
# it to C, compiled the code and wrapped it with Python
image = np.random.random((100, 100))
filt = np.random.random((10, 10))

In [28]:
%%timeit -n5
pass
res = fastfilter_2d(image, filt)

5 loops, best of 3: 1.68 ms per loop


In [29]:
%%timeit -n5
pass
filter2d(image, filt)

5 loops, best of 3: 550 ms per loop


In [33]:
@autojit
def generate_power_func(n):
    @jit(float_(float_))
    def nth_power(x):
        return x ** n

    # This is a native call
    print (nth_power(10))

    # Return closure and keep all cell variables alive
    return nth_power

for n in range(2, 5):
    func = generate_power_func(n)
    print ([func(x) for x in np.linspace(1.,2.,10.)])

AttributeError: Failed at object (analyzing bytecode)
'DataFlowAnalysis' object has no attribute 'op_LOAD_CLOSURE'